In [2]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

def get_x(path, width):
    """Gets the x value from the image filename"""
    return (float(int(path.split("_")[1])) - width/2) / (width/2)

def get_y(path, height):
    """Gets the y value from the image filename"""
    return (float(int(path.split("_")[2])) - height/2) / (height/2)

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        width, height = image.size
        x = float(get_x(os.path.basename(image_path), width))
        y = float(get_y(os.path.basename(image_path), height))
      
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset('dataset_xy', random_hflips=False)

print("Dataset created")

#############################################
# Split dataset into train and test sets
#############################################

test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

print(f"Dataset splitted into test ({test_percent}) and train ({1 - test_percent})")

#############################################
# Create data loaders to load data in batches
#############################################

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

print("Created data loaders to load data in batches")

#############################################
# Define Neural Network Model 
#############################################

# More details on ResNet-18 : https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
# More Details on Transfer Learning: https://www.youtube.com/watch?v=yofjFQddwHE 

model = models.resnet18(pretrained=True)

# ResNet model has fully connect (fc) final layer with 512 as ``in_features`` and we will be training for regression thus ``out_features`` as 1
# Finally, we transfer our model for execution on the GPU

model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

print("Neural network defined: resnet18")

#############################################
# Train Regression
#############################################

NUM_EPOCHS = 70
BEST_MODEL_PATH = 'best_steering_model_xy.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += float(loss)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss
        
print("Training ready")

Dataset created
Dataset splitted into test (0.1) and train (0.9)
Created data loaders to load data in batches


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/nelson/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


Neural network defined: resnet18
0.784753, 5.500496
1.624622, 1.208084
0.624026, 18.172390
0.200996, 12.848871
0.125467, 2.985002
0.112020, 0.453683
0.060057, 0.274478
0.074749, 0.090432
0.050649, 0.387439
0.046372, 0.262541
0.033214, 0.049774
0.024448, 0.048141
0.047531, 0.053305
0.034870, 0.035154
0.028830, 0.141492
0.040279, 0.064277
0.039372, 0.088067
0.028173, 0.076029
0.024259, 0.040744
0.014244, 0.077296
0.021646, 0.101710
0.026223, 0.059138
0.033173, 0.079881
0.013856, 0.031342
0.018522, 0.050789
0.041958, 0.049411
0.037880, 0.035152
0.068014, 0.046873
0.028280, 0.100623
0.043492, 0.064927
0.023100, 0.046138
0.027357, 0.058754
0.040823, 0.041786
0.030638, 0.033628
0.061644, 0.059996
0.037168, 0.084861
0.013792, 0.099750
0.032866, 0.044895
0.029486, 0.103327
0.021416, 0.066158
0.016322, 0.047163
0.025563, 0.054298
0.021803, 0.093469
0.034151, 0.037917
0.016867, 0.071083
0.028673, 0.078881
0.013592, 0.060048
0.007643, 0.058922
0.015824, 0.042085
0.021162, 0.049677
0.013534, 0.049